In [1]:
# Generates test data and calculates success rates for BMA using 95% credible intervals, with the new numpy GTC
# Saves results in this folder.

# Can update the participants function to generate success rates for different combinations of biased and unbiased
# participants. See txt file in same folder as this to find other versions of the 'participants' function.

code_version = 1

In [2]:
from GTC import *
import math
import xlwt
from scipy import stats
import numpy as np
import operator as op
import itertools
import random
from scipy.stats import chi2
import scipy

In [3]:
def LChS(y,N):
    # Largest coherent subset
    # To store maximum subset size
    lchs = [0,0]
    
    # Calculate the values of alpha_star and z
    alpha_star = 2*(0.05/(N*(N-1)))
    z = stats.norm.ppf(1-alpha_star/2)
    
    # Initialise I to store results. I will contain a 1 in the i,j th position if NMIs i,j are not equivalent, and a 0 if
    # they are equivalent
    I = np.ones(([N,N]))
        
    # Iterate through the NMIs twice
    for i in range(0,N):
        for k in range(0,N):  
                
            if i != k:
                # If i,k correspond to different NMIs
                    
                # Calculate the lower and upper bounds to t
                lower = value(y[i]) - value(y[k]) - z*np.sqrt(uncertainty(y[i])**2 + uncertainty(y[k])**2)
                upper = value(y[i]) - value(y[k]) + z*np.sqrt(uncertainty(y[i])**2 + uncertainty(y[k])**2)
                
                # If lower < 0 < upper update I to show i,k are equivalent
                if lower < 0 and upper > 0:
                    I[i,k] = 0
                
            else:
                # If we are considering the same NMI, then it is equivalent to itself so set I to 0
                I[i,k] = 0
    
    
    # Determine the largest coherent subset for each column
    # Create dictionaries to store values, equiv for the NMIs which are equivalent pairwise
    equiv = {}

    for i in range(0,N):
        # Iterate through the rows in the matrix
        equiv[i] = []

        for k in range(0,N):
            # Iterate through the columns

            if I[i][k]==0:
                # If 0, add to a
                equiv[i].append(k)
        
    lchs = 0 # to store maximum subset size
        
    for i in range(0,N):
        # Iterate through the rows of the matrix
        s = []
        s.append(i)
        rows = list(range(0,N))
        stop = False
    
        while stop == False:
            # Until we finish going through all rotations of the list
                
            # Iterate through the rows of the matrix and if the row is equivalent to all elements in s, add row j to s
            for k in rows:
                test = 0
                for v in s:
                    if k not in equiv[v]:
                        test += 1
                if test == 0:
                    s.append(k)
                
            # Get rid of any repeats in s, and if its length is longer than the largest so far, save it
            s2 = set(s)
            if len(s2) >= lchs:
                lchs = len(s2)
        
            # Move all the elements in the list rows around
            rows = rows[1:] + [rows[0]]
                
            s = []
            s.append(i)
        
            # If we have tried all possible starting elements, get out of loop
            if rows[0] == 0:
                stop = True  
    
    return(lchs)

In [4]:
def BMA(y,N,t):
    # Bayesian model averaging for integrating
    import operator as op
    import itertools
    
    S_N = set((i for i in range(0,N)))
    
    # List of all subsets of size t in set s of size N
    ss = [set(i) for i in itertools.combinations(S_N, t)]
    Q = len(ss)
    
    # Convert to array of arrays
    St =np.array([list(ss[i]) for i in range(0,Q)])
    
    P_P = np.zeros([Q],dtype = object)
    d_all = np.zeros([N,Q],dtype = object)
    theta_all = np.zeros([Q],dtype = object)
    alpha_all = np.zeros([N,Q],dtype = object)
    var_mu_M = np.zeros([Q],dtype = float)
    var_alpha_M = np.zeros([N,Q],dtype = float)
    pdf = np.zeros([Q], dtype = float)
    w_all = np.zeros([Q,N])
    
    for m in range(0,Q):
        # Weighting vector for set of participants with zero bias
        w_s = np.zeros([N,])
        
        for i in range(0,t):      
            w_s[St[m,i]] = 1.   
        w_all[m,] = w_s
        
        # Inverse uncerts for only non-zero participants
        wa = np.array([(1-w_s[i])/(uncertainty(y[i])**2) for i in range(0,N)])
        
        # Inverse uncerts for zero-bias participants
        wp = w_s/(y.uncertainty()**2)         
        
        # Weights for zero-bias participants
        w = wp/sum(wp) 
        
        # Equation 12
        var_mu_M[m] = 1./sum(wp)
        
        # Equation 11
        theta_all[m] = var_mu_M[m]*np.dot(wp,y)
        
        # d is used to calculate probabilities
        d_all[:,m] = y - theta_all[m]
    
        # But biases should be zero for the participants with zero bias
        # Random variable alpha_i, Equation 13
        alpha_all[:,m] = np.array([(1.-w_s[i])*(y[i]-theta_all[m]) for i in range(0,N)])
        
        # Variance of random variable alpha_i under model M_l, Equation 14
        var_alpha_M[:,m] = np.array([(1.-w_s[i])*(uncertainty(y[i])**2+var_mu_M[m]) for i in range(0,N)])
        
        # Equation 6
        P1_e = np.array([exp(-d_all[i,m]**2*wp[i]/2.) for i in range(0,N)])
        P1 = np.product(P1_e)
        P2 = np.sqrt(var_mu_M[m])  
        P3 = np.sqrt(np.product(np.compress(wp.flat!=0, wp.flat)))
        
        # Proportional probability of each model
        P_P[m] = P1*P2*P3
    
    
    # Probability from Equation 6
    S_P = np.sum(P_P)
    P = np.array([P_P[m]/S_P for m in range(0,Q)])
    
    # Equation 7
    mu = np.sum(theta_all*P)
    
    # Equation 8
    alpha = np.array([(np.sum(alpha_all[i]*P)) for i in range(0,N)])
    
    # Equation 9
    v_m = np.array([(var_mu_M[m]+(value(theta_all[m])-value(mu))**2)*P[m] for m in range(0,Q)])
    
    # Equation 10
    v_a=np.array([[(var_alpha_M[i,m]+(value(alpha_all[i,m])-value(alpha[i]))**2)*P[m] for i in range(0,N)]for m in range(0,Q)])
    
    unc_alpha = np.array([np.sqrt(value(np.sum(v_a[:,i]))) for i in range(0,N)])
    
    alpha = la.uarray([ureal(value(alpha[i]),value(unc_alpha[i])) for i in range(0,len(alpha))])
    
    return(theta_all, var_mu_M, P, Q, w_all, alpha, unc_alpha)

In [5]:
def pdf(a, theta_all, var_mu_M, P, Q, y, w_all, i):
    # Sum up gaussian pdfs for alpha
    pdf_sum = 0
    for l in range(0,Q):
        w_l = w_all[l,]
        if w_l[i] != 1:
            pdf_sum += value(P[l]) * np.exp(-(a-(value(y[i])-value(theta_all[l])))**2/(2*(uncertainty(y[i])**2+var_mu_M[l]))) / np.sqrt(2*np.pi*(uncertainty(y[i])**2+var_mu_M[l]))
    return pdf_sum

In [6]:
def delta_function(Q,w_all,i,P):
    # Sum up the delta functions for alpha
    delta_sum = 0
    for l in range(0,Q):
        w_l = w_all[l,]
        if w_l[i] == 1:
            delta_sum += value(P[l])
    return delta_sum

In [7]:
def search_function(a,mu,sigma,theta_all,var_mu_M,P,Q,y,w_all,i):
    # Calculate the area under the probability density function between mu-a and mu+a
    if mu-a > 0:
        area = scipy.integrate.quad(pdf,mu-a,mu+a,args=(theta_all,var_mu_M,P,Q,y,w_all,i))[0]
    elif mu+a < 0:
        area = scipy.integrate.quad(pdf,mu-a,mu+a,args=(theta_all,var_mu_M,P,Q,y,w_all,i))[0]
    else:
        # If it crosses zero, add delta function
        delta_sum = delta_function(Q,w_all,i,P)
        area = delta_sum + scipy.integrate.quad(pdf,mu-a,mu+a,args=(theta_all,var_mu_M,P,Q,y,w_all,i))[0]
    return area

def bisection(f,N,tol,theta_all,var_mu_M,P,Q,y,w_all,i, alpha_mu, alpha_var):
    # Use above function to determine the value of a which gives a 95% confidence interval
    mu = value(alpha_mu[i])
    sigma = value(alpha_var[i])

    a = 0
    b = 20
    c = (b-a)/2

    for n in range(1,N+1):
        if 0.95 - f(c,mu,sigma,theta_all,var_mu_M,P,Q,y,w_all,i) < -tol:
            if (b-a)/2 < tol:
                c = c+0.1
                return c, f(c,mu,sigma,theta_all,var_mu_M,P,Q,y,w_all,i)
            
            b = c*1.
            c = a + (b-a)/2
            
        elif 0.95 - f(c,mu,sigma,theta_all,var_mu_M,P,Q,y,w_all,i) > tol:
            if (b-a)/2 < tol:
                c = c+0.1
                return c, f(c,mu,sigma,theta_all,var_mu_M,P,Q,y,w_all,i)
            a = c*1.
            c = a + (b-a)/2
        
        elif b-a < tol:
            c = c+0.1
            return c, f(c,mu,sigma,theta_all,var_mu_M,P,Q,y,w_all,i)
        
        else:
            return c, f(c,mu,sigma,theta_all,var_mu_M,P,Q,y,w_all,i)
    
    c = c+0.1
    return c, f(c,mu,sigma,theta_all,var_mu_M,P,Q,y,w_all,i)

In [8]:
# One biased participant

def participants(N,f,g,d):
    # Generate a participant set, run the methods and return the required variables to determine success
    # Takes in: N = number of participants
    #           f = uncertainty of the biased lab
    #           g = size of bias
    #           d = degrees of freedom of Chi Square distribution used to generated uncertainties
    # Returns:  y = uncertain array containing N participants, the first of which has a bias defined by f,g
    
    unc = np.array([np.random.chisquare(d)/d for i in range(0,N-1)])
    unc = np.insert(unc, 0, f) # Set the first lab to have f=1

    # Generate errors from a Normal (0,unc) distribution
    err = np.array([random.normalvariate(0,unc[i]) for i in range(1,N)])
    err = np.insert(err, 0, [random.normalvariate(0,unc[0])+g]) # Set the first lab to have a bias of 6*unc

    # Put 12 participants into an uncertain number array
    y = la.uarray([ureal(err[i],unc[i]) for i in range(0,N)])
    
    n_biased = 1
    
    return(y, n_biased)

In [9]:
def simulate_data(T,N,f,g,df):
    # Generate datasets, and tests them to see how many labs fail using each method
    
    # Create array in which to store the failure rates, and m
    fail = np.zeros((N))
    m_sum = 0
    
    # Initialise stop (to break out of while loop), t_ave
    stop = T * 1.
    t_ave = 0

    while stop > 0:
        print(stop)
        # Create a set of N participants
        [y, n_bias] = participants(N,f,g,df)
        
        # Find size of largest coherent subset
        lchs = LChS(y,N)
        
        # Set m to be lchs - 1
        m = lchs - 1
        m_sum += m
        
        # Run participant set through BMA
        [theta_all, var_mu_M, P, Q, w_all, alpha, unc_alpha] = BMA(y,N,m)
        
        # Create temporary arrays to store results
        k_temp = np.zeros(N)
        coverage_temp = np.zeros(N)
        
         # Calculate coverage factor for each participant
        for j in range(0,N):           
            # Calculate delta function value
            delta = delta_function(Q,w_all,j,P)

            # Calculate k and coverage value
            [k_temp[j], coverage_temp[j]] = bisection(search_function,100,0.001,theta_all,var_mu_M,P,Q,y,w_all,j,alpha,unc_alpha)

            # Determine failure using k
            if abs(alpha[j]) > k_temp[j]:
                fail[j] += 1
        
        # Update stop
        stop -= 1
    
    # Calculate fail rates by dividing the number of labs which failed by the number of trials
    fail_rate = fail/T
    m_ave = m_sum / T
    
    return(fail_rate, m_ave, n_bias)

In [13]:
N = 12
T = 1
df = 4

g_all = [2.5]
f_all = [1]

for f in f_all:
    for g in g_all:

        # Initliase spreadsheet to save results
        wb = xlwt.Workbook()
        ws = wb.add_sheet('Fail rate')

        # Generate failure rates
        [fail_rates, m_ave, n_biased] = simulate_data(T,N,f,g,df)
        
        if n_biased == 1:
            ws.write(0,0,'Biased')
        elif n_biased == 2:
            ws.write(0,0,'Biased')
            ws.write(0,1,'Biased')

        # Write results to spreadsheet
        for i in range(0,N):
            ws.write(1,i,fail_rates[i])

        ws.write(0, 13, 'f')
        ws.write(0, 14, f)
        ws.write(1, 13, 'g')
        ws.write(1, 14, g)
        ws.write(2, 13, 'Average m')
        ws.write(2, 14, m_ave)
        ws.write(3, 13, 'Code version')
        ws.write(3, 14, code_version)

        wb.save('2bias,+-,BMA-lchs-1,f='+str(f)+',g='+str(g)+'.xls')

1.0
